In [ ]:
import os, sys

p = "/kaggle/input/sage-zrok-token/.zrok_api_key"
zrok_token = None

if os.path.isfile(p):
    with open(p, "r", encoding="utf-8", errors="ignore") as f:
        zrok_token = f.read().strip()

if not zrok_token:
    print("❌ Token not found or empty:", p)
    sys.exit(1)

In [ ]:
!pip install controlnet_aux==0.0.9 --no-deps

In [ ]:

###########################
# CONTROLNET AUX DETECTORS
###########################

#!pip install controlnet_aux>=0.0.8

from PIL import Image
import requests
from io import BytesIO

from controlnet_aux import OpenposeDetector, HEDdetector, MLSDdetector, MidasDetector, CannyDetector

# pick the HF repo used by controlnet-aux examples:
open_pose = OpenposeDetector.from_pretrained("lllyasviel/Annotators")
hed = HEDdetector.from_pretrained("lllyasviel/Annotators")
mlsd = MLSDdetector.from_pretrained("lllyasviel/Annotators")
midas = MidasDetector.from_pretrained("lllyasviel/Annotators")
canny = CannyDetector()

In [ ]:
# ---------------------------
# CONTROLNET MAP FUNCTIONS
# ---------------------------

from PIL import Image
from io import BytesIO

def generate_pose_map(img: Image.Image, hand_and_face: bool = True) -> Image.Image:
    return open_pose(img, hand_and_face=hand_and_face)

def generate_hed_map(img: Image.Image) -> Image.Image:
    return hed(img)

def generate_canny_map(img: Image.Image) -> Image.Image:
    return canny(img)

def generate_midas_map(img: Image.Image) -> Image.Image:
    return midas(img)

def generate_mlsd_map(img: Image.Image) -> Image.Image:
    return mlsd(img)

In [ ]:

# --------------------------
# FastAPI app init
# --------------------------

from fastapi import FastAPI
import nest_asyncio
import uvicorn

# Assume generate_txt2img, generate_img2img, pipe_txt2img, pipe_img2img are already defined
app = FastAPI()
nest_asyncio.apply()  # allow running uvicorn in Colab
print("FastAPI app initialized.")

In [ ]:

# ---------------------------
# MAP ENDPOINTS
# ---------------------------

from fastapi import UploadFile, File
from fastapi.responses import StreamingResponse, JSONResponse

def image_response(pil_image: Image.Image) -> StreamingResponse:
    """Helper to return PIL Image as PNG in response."""
    buffer = BytesIO()
    pil_image.save(buffer, format="PNG")
    buffer.seek(0)
    return StreamingResponse(buffer, media_type="image/png")


@app.post("/map/pose")
async def map_pose(file: UploadFile = File(...), hand_and_face: bool = True):
    try:
        img = Image.open(BytesIO(await file.read())).convert("RGB")
        result = generate_pose_map(img, hand_and_face=hand_and_face)
        return image_response(result)
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


@app.post("/map/hed")
async def map_hed(file: UploadFile = File(...)):
    try:
        img = Image.open(BytesIO(await file.read())).convert("RGB")
        result = generate_hed_map(img)
        return image_response(result)
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


@app.post("/map/canny")
async def map_canny(file: UploadFile = File(...)):
    try:
        img = Image.open(BytesIO(await file.read())).convert("RGB")
        result = generate_canny_map(img)
        return image_response(result)
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


@app.post("/map/midas")
async def map_midas(file: UploadFile = File(...)):
    try:
        img = Image.open(BytesIO(await file.read())).convert("RGB")
        result = generate_midas_map(img)
        return image_response(result)
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


@app.post("/map/mlsd")
async def map_mlsd(file: UploadFile = File(...)):
    try:
        img = Image.open(BytesIO(await file.read())).convert("RGB")
        result = generate_mlsd_map(img)
        return image_response(result)
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

In [ ]:
# Download zrok v1.1.3 (latest)
!wget https://github.com/openziti/zrok/releases/download/v1.1.3/zrok_1.1.3_linux_amd64.tar.gz
!tar -xzf zrok_1.1.3_linux_amd64.tar.gz
!chmod +x zrok

In [ ]:

# Enable (automatic migration from 0.4)
!./zrok enable --headless "$zrok_token"

# Use the agent for better process management
#!./zrok agent start &
#!./zrok share public localhost:8000 --headless

In [ ]:
#!./zrok disable

In [ ]:

import uvicorn
import threading

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Start in background thread
threading.Thread(target=run_uvicorn, daemon=True).start()

In [ ]:
import subprocess
import re
import time

def start_zrok_tunnel(port=8000):
    # Start the tunnel
    process = subprocess.Popen([
        "./zrok", "share", "public", f"localhost:{port}", "--headless"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Give it a moment to start
    time.sleep(3)

    # Check agent status to get the URL
    status_process = subprocess.run([
        "./zrok", "agent", "status"
    ], capture_output=True, text=True)

    print("Agent Status:")
    print(status_process.stdout)

    return process

# Start the tunnel
tunnel_process = start_zrok_tunnel(8000)
print("Zrok tunnel started! Check the agent status above for your public URL.")

In [ ]:
!./zrok overview

In [ ]:
import time

print("Server and zrok tunnel are running. Keeping the notebook alive...")

try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("Shutting down.")